In [84]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

database_file = "data/glider_classification.db"
download_folder = "data/pdf"

air_turquoise_page_size=25
air_turquise_base_url = 'https://para-test.com'
air_turquoise_links = '{air_turquise_base_url}/component/jak2filter/?Itemid=114&issearch=1&isc=1&category_id=11&xf_3_txt=B&ordering=publishUp&orders[publishUp]=rpublishUp&orders[date]=date&start={index}'
# C : https://para-test.com/component/jak2filter/?Itemid=114&issearch=1&isc=1&category_id=11&xf_3_txt={class}&ordering=publishUp&orders[publishUp]=rpublishUp&orders[date]=date&start=0

# set up DB (once)
conn = sqlite3.connect(database_file)
cur = conn.cursor()
cur.execute('''CREATE TABLE IF NOT EXISTS air_turquoise_reports
               (report_date, item_name, report_link, download_link)''')
conn.commit()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())
conn.close()    
    

[('air_turquoise_reports',)]


In [36]:
# get base data - once
data = []
for page in range(0,10):
    index = air_turquoise_page_size * page
    url = eval(f"f'{air_turquoise_links}'") # auto
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.text, 'html.parser')
        #print(soup.prettify)        
        tbl = soup.find('table',{"class":"itemList"})
        assert(len(tbl.findAll('tr'))-1,air_turquoise_page_size)
        for tr in tbl.findAll('tr')[1:]:
            #print(tr.prettify())
            report_date = tr.find('td',{"class":"catItemDateCreated"}).text.strip()
            report_anchor = tr.find('a')
            report_link, item_name = report_anchor['href'], report_anchor.text.strip()
            #print((report_date, item_name, report_link))
            data.append((report_date, item_name, report_link))
    except requests.exceptions.HTTPError as err:
        print(err)
        
pd.DataFrame(data)

25
25
25
25
25
25
25
25
25
25


,0,1,2
0,05.05.2023,U-Turn Lightning 2 M,/reports/item/5258-u-turn-lightning-2-m
1,02.05.2023,Supair LEAF 3 Light XXS,/reports/item/5254-supair-leaf-3-light-xxs
2,02.05.2023,NOVA Vertriebsgesellschaft Mentor 7 M,/reports/item/5253-nova-vertriebsgesellschaft-...
3,02.05.2023,NOVA Vertriebsgesellschaft Mentor 7 S,/reports/item/5252-nova-vertriebsgesellschaft-...
4,02.05.2023,NOVA Vertriebsgesellschaft Mentor 7 XS,/reports/item/5251-nova-vertriebsgesellschaft-...
...,...,...,...
245,13.02.2020,Supair Leaf 2 light ML,/reports/item/4370-supair-leaf-2-light-ml
246,13.02.2020,Supair Leaf 2 light M,/reports/item/4369-supair-leaf-2-light-m
247,13.02.2020,Supair Leaf 2 light S,/reports/item/4368-supair-leaf-2-light-s
248,13.02.2020,Supair Leaf 2 light XS,/reports/item/4367-supair-leaf-2-light-xs


In [37]:
# add to DB  
conn = sqlite3.connect(database_file)
cur = conn.cursor()
#cur.execute("delete from air_turquoise_reports")
#cur.executemany("insert into air_turquoise_reports values (?, ?, ?, null)", data)
    
conn.commit()
conn.close() 

In [75]:
with sqlite3.connect(database_file) as conn:
    df = pd.read_sql_query("""select * 
                            from air_turquoise_reports 
                            where download_link IS NULL 
                            order by report_link""",conn)

    df.head()

    # fetch download urls
    for report in df.itertuples():
        #print(report)
        url = f"{air_turquise_base_url}{report.report_link}"
        #print(url)
        try:
            resp = requests.get(url)
            resp.raise_for_status()
            soup = BeautifulSoup(resp.text, 'html.parser')
            #print(soup.prettify)
            a = soup.find("a",{"title":"Flight report"})
            if not a:
                a = soup.find("a",{"title":"Flight report trimmer closed"}) # tandems?
            if not a:
                a = soup.find("a",{"title":" Flight report"})
            if not a:
                a = soup.find("a",{"title":"Flight report "})  
             
            if a:
                #print(a.prettify())
                download_link = a['href']
                print(download_link)
                cur = conn.cursor()
                cur.execute("""UPDATE air_turquoise_reports
                                SET download_link = ?
                                WHERE download_link IS NULL AND report_link = ?
                """, (download_link,report.report_link))
                conn.commit()
            else:
                print(url)

        except requests.exceptions.HTTPError as err:
            print(err)    
    


https://para-test.com/reports/item/4655-swing-flugsportgeraete-sting-rs-24
https://para-test.com/reports/item/4656-swing-flugsportgeraete-sting-rs-26
https://para-test.com/reports/item/4657-swing-flugsportgeraete-sting-rs-28
https://para-test.com/reports/item/4658-swing-flugsportgeraete-sting-rs-31
/reports/item/download/31449_db6b9589ac5c2b04542986c9c021895a
/reports/item/download/31621_dd2260ca6e3f043980a4aa5055bfeb4a
https://para-test.com/reports/item/4836-windtech-paragliders-ru-bi-3-39
/reports/item/download/32456_332e7c7e618986b3909f0bb963c8f50f
/reports/item/download/32814_3e2336cc9d45afd7c7ddcb28f7ee5205
/reports/item/download/34286_f43f70bd63c5cb40a68d6d058826f703
/reports/item/download/34527_2a173bdfdb4c6d7bdbf36d68ae8cb16b


In [77]:
with sqlite3.connect(database_file) as conn:
    df = pd.read_sql_query("""select * from air_turquoise_reports where download_link is null order by report_link""",conn)

df.head(20)

,report_date,item_name,report_link,download_link
0,06.05.2021,Swing Flugsportgeräte Sting RS 24,/reports/item/4655-swing-flugsportgeraete-stin...,None
1,06.05.2021,Swing Flugsportgeräte Sting RS 26,/reports/item/4656-swing-flugsportgeraete-stin...,None
2,06.05.2021,Swing Flugsportgeräte Sting RS 28,/reports/item/4657-swing-flugsportgeraete-stin...,None
3,06.05.2021,Swing Flugsportgeräte Sting RS 31,/reports/item/4658-swing-flugsportgeraete-stin...,None
4,28.01.2022,Windtech Paragliders Ru-Bi 3 39,/reports/item/4836-windtech-paragliders-ru-bi-...,None


In [ ]:
from os.path import exists

# download all pdf
with sqlite3.connect(database_file) as conn:
    df = pd.read_sql_query("""select * from air_turquoise_reports where download_link is not null order by report_link""",conn)
    
for report in df.itertuples():
    fname = f"{download_folder}/{'_'.join(report.item_name.split())}.pdf"
    url = f"{air_turquise_base_url}{report.download_link}"    
    if not exists(fname):
        print(fname,url)
        r = requests.get(url, allow_redirects=True)
        open(fname, 'wb').write(r.content) 


